In [11]:
import requests
import json
from time import time, sleep 
from openai import OpenAI 
import os
from dotenv import load_dotenv
import pandas as pd 
from pathlib import Path 
# Load environment variables from .env file
load_dotenv()

# Replace with your actual DeepSeek API endpoint and API key
DEEPSEEK_API_URL = "https://api.deepseek.com"
DEEPSEEK_API_KEY = os.getenv('DEEPSEEK_API_KEY')

In [2]:
deepseek_client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_API_URL)

In [3]:
def generate_custom_prompt(character):
    """
    Generates a custom prompt for the DeepSeek API based on your template.

    Args:
        character (str): The Chinese character to query.

    Returns:
        str: A formatted prompt for the API.
    """
    prompt = f"""
    You are an expert in Chinese language. 
    Generate a holistic view on the Chinese character '{character}' 
    in terms of the following attributes:

    1. 含义 (meaning): Explain its meaning.
    2. 字形 (character structure): Describe its structure.
    3. 读音 (pronunciation): Provide its pronunciation.
    4. 字源 (etymology): Explain its origin.
    5. 含此字的字 (composite characters): List characters that contain '{character}'.
    6. 同音字 (homophone characters): List other characters that sound like '{character}'.
    7. 常用词组 (common phrases): Provide common phrases containing '{character}' (often 2-characters).
    8. 成语 (idioms): List idioms containing '{character}' (often 4-characters).
    9. 例句 (example sentences): Provide example short sentences or famous quotes using '{character}'.
    10. 短故事 (short stories): Share 1-2 short stories related to '{character}'.
    11. 诗词 (poetry): Include 1-2 famous poems containing and describing '{character}'.
    12. 图片 (images): Suggest simple images illustrating '{character}'.
    13. 音频 (audio): Suggest short audio clips illustrating '{character}'.
    14. 视频 (video): Suggest short video clips illustrating '{character}'.
    15. 电影 (movies): List famous movies related to '{character}' or movie titles containing '{character}'.
    16. 参考资料 (references): Provide additional popular reference materials.
    17. 有趣网站 (interesting websites): Suggest popular websites related to '{character}'.

    Format the output as a valid JSON object.
    """
    return prompt.strip()

In [4]:
def extract_json_string(response):
    # Find the start and end of the JSON string
    start = response.find("```json") + len("```json")
    end = response.find("```", start)

    # Extract the JSON string and strip any leading/trailing whitespace
    json_str = response[start:end].strip()

    return json_str

In [5]:
def call_deepseek_api(prompt, system_message = "You are a helpful assistant", client=None):
    if client is None:
        client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_API_URL)
    ts_start = time()
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
        stream=False
    )
    ts_stop = time()

    return response.choices[0].message.content, (ts_stop-ts_start)

In [6]:
def main(character = "子", test_flag=False, debug_flag=False):
    # Call the DeepSeek API
    if test_flag:
        prompt = "What is 10! in math"
    else: 
        prompt = generate_custom_prompt(character)
    if debug_flag: 
        print(f"Prompt:\n{prompt}\n")

    response, ts_delta = call_deepseek_api(prompt, client=deepseek_client)
    json_str = extract_json_string(response)
    json_obj = json.loads(json_str)
    json_out = json.dumps(json_obj, indent=2, ensure_ascii=False)
    file_json = f"deepseek/{character}-1.json"
    if debug_flag: 
        print(f"API call completed in {ts_delta} sec, \n Response (JSON):\n {json_out} \n written to {file_json}")
    else: 
        print(f"API call for {character} completed in {ts_delta} sec, \n output written to {file_json}")
    with open(file_json, "w", encoding="utf-8") as f:
        f.write(json_out)
            

# if __name__ == "__main__":
#     main()

In [7]:
file_csv_in = "elemental_zi_no_radical.csv"
file_csv_in = "HSK-3000-20250119.csv"

df = pd.read_csv(file_csv_in)

In [8]:
df.columns 

Index(['zi', 'layer', 'is_picto', 'is_radical', 'fib_num', 'desc_en'], dtype='object')

In [9]:
zis = df["zi"].to_list()

In [10]:
len(zis), zis[:10]

(3890, ['上', '才', '木', '山', '也', '于', '中', '贝', '比', '卜'])

In [12]:
processed = []
for zi in zis[2:]:
    file_json = f"deepseek/{zi}-1.json"
    if Path(file_json).exists():
        print(f"Skip {zi}")
        continue
    try:
        main(character = zi)
        processed.append(zi)
    except Exception as e: 
        print(f"Failed to process {zi}:\n str(e)")

Skip 木
Skip 山
Skip 也
Skip 于
Skip 中
Skip 贝
Skip 比
Skip 卜
API call for 仇 completed in 22.706754207611084 sec, 
 output written to deepseek/仇-1.json
Skip 川
API call for 从 completed in 22.04313349723816 sec, 
 output written to deepseek/从-1.json
Skip 刀
Skip 丁
Skip 儿
API call for 凡 completed in 20.563872814178467 sec, 
 output written to deepseek/凡-1.json
Skip 方
API call for 凤 completed in 22.713053464889526 sec, 
 output written to deepseek/凤-1.json
Skip 弓
API call for 化 completed in 21.676973581314087 sec, 
 output written to deepseek/化-1.json
Skip 火
API call for 计 completed in 23.25997543334961 sec, 
 output written to deepseek/计-1.json
Skip 井
Skip 了
Skip 力
Skip 马
Skip 毛
Skip 门
Skip 乃
Skip 牛
Skip 女
API call for 仆 completed in 17.477332830429077 sec, 
 output written to deepseek/仆-1.json
Skip 千
Skip 区
Skip 日
Skip 三
API call for 什 completed in 22.318782806396484 sec, 
 output written to deepseek/什-1.json
Skip 升
Skip 尸
Skip 士
Skip 氏
Skip 水
Skip 土
Skip 万
Skip 亡
API call for 卫 completed in 24

API call for 叼 completed in 17.287044048309326 sec, 
 output written to deepseek/叼-1.json
API call for 叮 completed in 19.040441036224365 sec, 
 output written to deepseek/叮-1.json
API call for 动 completed in 22.692590475082397 sec, 
 output written to deepseek/动-1.json
API call for 队 completed in 22.92410111427307 sec, 
 output written to deepseek/队-1.json
API call for 犯 completed in 25.588223695755005 sec, 
 output written to deepseek/犯-1.json
API call for 功 completed in 23.211514711380005 sec, 
 output written to deepseek/功-1.json
Skip 瓜
Skip 禾
Skip 乎
API call for 幻 completed in 25.408817768096924 sec, 
 output written to deepseek/幻-1.json
Skip 目
Skip 击
API call for 记 completed in 26.281931400299072 sec, 
 output written to deepseek/记-1.json
Skip 甲
API call for 旧 completed in 23.30423665046692 sec, 
 output written to deepseek/旧-1.json
Skip 另
Skip 矛
API call for 们 completed in 32.80459403991699 sec, 
 output written to deepseek/们-1.json
API call for 灭 completed in 23.504780292510986 

API call for 厌 completed in 28.68339252471924 sec, 
 output written to deepseek/厌-1.json
API call for 亦 completed in 20.837068796157837 sec, 
 output written to deepseek/亦-1.json
API call for 在 completed in 27.66297173500061 sec, 
 output written to deepseek/在-1.json
API call for 优 completed in 34.467397689819336 sec, 
 output written to deepseek/优-1.json
API call for 屿 completed in 27.84468698501587 sec, 
 output written to deepseek/屿-1.json
API call for 再 completed in 25.469910860061646 sec, 
 output written to deepseek/再-1.json
API call for 早 completed in 22.665499210357666 sec, 
 output written to deepseek/早-1.json
API call for 企 completed in 32.88684582710266 sec, 
 output written to deepseek/企-1.json
API call for 贞 completed in 20.36717677116394 sec, 
 output written to deepseek/贞-1.json
API call for 争 completed in 29.657585859298706 sec, 
 output written to deepseek/争-1.json
API call for 芝 completed in 19.585246324539185 sec, 
 output written to deepseek/芝-1.json
API call for 执 

API call for 克 completed in 20.530669450759888 sec, 
 output written to deepseek/克-1.json
API call for 困 completed in 23.650095224380493 sec, 
 output written to deepseek/困-1.json
API call for 李 completed in 21.355729341506958 sec, 
 output written to deepseek/李-1.json
Skip 里
API call for 励 completed in 25.06001353263855 sec, 
 output written to deepseek/励-1.json
API call for 丽 completed in 25.28594994544983 sec, 
 output written to deepseek/丽-1.json
API call for 连 completed in 21.732396125793457 sec, 
 output written to deepseek/连-1.json
Skip 麦
API call for 农 completed in 22.088674783706665 sec, 
 output written to deepseek/农-1.json
API call for 却 completed in 25.041456937789917 sec, 
 output written to deepseek/却-1.json
API call for 时 completed in 19.938356637954712 sec, 
 output written to deepseek/时-1.json
API call for 寿 completed in 27.30627202987671 sec, 
 output written to deepseek/寿-1.json
API call for 束 completed in 22.037803411483765 sec, 
 output written to deepseek/束-1.json

API call for 旷 completed in 18.433027029037476 sec, 
 output written to deepseek/旷-1.json
API call for 来 completed in 19.709870100021362 sec, 
 output written to deepseek/来-1.json
API call for 劳 completed in 23.391037702560425 sec, 
 output written to deepseek/劳-1.json
API call for 两 completed in 21.58121371269226 sec, 
 output written to deepseek/两-1.json
API call for 芦 completed in 24.795020818710327 sec, 
 output written to deepseek/芦-1.json
API call for 妈 completed in 30.300395727157593 sec, 
 output written to deepseek/妈-1.json
API call for 买 completed in 25.190552711486816 sec, 
 output written to deepseek/买-1.json
API call for 忙 completed in 32.09927558898926 sec, 
 output written to deepseek/忙-1.json
API call for 男 completed in 38.00510883331299 sec, 
 output written to deepseek/男-1.json
API call for 扭 completed in 23.171266317367554 sec, 
 output written to deepseek/扭-1.json
API call for 抛 completed in 31.754472970962524 sec, 
 output written to deepseek/抛-1.json
API call for 

...

API call for 豁 completed in 14.73069167137146 sec, 
 output written to deepseek/豁-1.json
API call for 靡 completed in 14.262117385864258 sec, 
 output written to deepseek/靡-1.json
API call for 甭 completed in 11.964486360549927 sec, 
 output written to deepseek/甭-1.json
API call for 亢 completed in 12.829375505447388 sec, 
 output written to deepseek/亢-1.json
API call for 馈 completed in 18.52018666267395 sec, 
 output written to deepseek/馈-1.json
API call for 饪 completed in 13.80384087562561 sec, 
 output written to deepseek/饪-1.json
API call for 哇 completed in 18.119768142700195 sec, 
 output written to deepseek/哇-1.json
API call for 惟 completed in 14.465081453323364 sec, 
 output written to deepseek/惟-1.json
API call for 墟 completed in 16.47083616256714 sec, 
 output written to deepseek/墟-1.json
API call for 铀 completed in 10.743938684463501 sec, 
 output written to deepseek/铀-1.json
Skip 曰
API call for 瞩 completed in 19.95937418937683 sec, 
 output written to deepseek/瞩-1.json
API call

API call for 獄 completed in 15.852420568466187 sec, 
 output written to deepseek/獄-1.json
API call for 鴦 completed in 15.519639492034912 sec, 
 output written to deepseek/鴦-1.json
API call for 鷖 completed in 11.854527711868286 sec, 
 output written to deepseek/鷖-1.json
API call for 䳁 completed in 14.723126411437988 sec, 
 output written to deepseek/䳁-1.json
API call for 鴜 completed in 10.867718935012817 sec, 
 output written to deepseek/鴜-1.json
API call for 鸔 completed in 9.451560497283936 sec, 
 output written to deepseek/鸔-1.json
API call for 鷟 completed in 13.360710859298706 sec, 
 output written to deepseek/鷟-1.json
API call for 䳘 completed in 16.28658938407898 sec, 
 output written to deepseek/䳘-1.json
API call for 達 completed in 22.619107961654663 sec, 
 output written to deepseek/達-1.json
API call for 羋 completed in 12.063305616378784 sec, 
 output written to deepseek/羋-1.json
API call for 羣 completed in 20.088157653808594 sec, 
 output written to deepseek/羣-1.json
API call for

API call for 敵 completed in 21.34236168861389 sec, 
 output written to deepseek/敵-1.json
API call for 𠛰 completed in 15.751927375793457 sec, 
 output written to deepseek/𠛰-1.json
API call for 㓝 completed in 21.244171857833862 sec, 
 output written to deepseek/㓝-1.json
API call for 𩮜 completed in 14.42268705368042 sec, 
 output written to deepseek/𩮜-1.json
API call for 𢖫 completed in 23.86559510231018 sec, 
 output written to deepseek/𢖫-1.json
API call for 𧗳 completed in 15.315049409866333 sec, 
 output written to deepseek/𧗳-1.json
API call for 詞 completed in 22.549527645111084 sec, 
 output written to deepseek/詞-1.json
API call for 琂 completed in 11.97267770767212 sec, 
 output written to deepseek/琂-1.json
API call for 𠶷 completed in 13.590705394744873 sec, 
 output written to deepseek/𠶷-1.json
API call for 𧥣 completed in 13.44517731666565 sec, 
 output written to deepseek/𧥣-1.json
API call for 譻 completed in 14.175955295562744 sec, 
 output written to deepseek/譻-1.json
API call for 𧭭 

API call for 辦 completed in 17.058205604553223 sec, 
 output written to deepseek/辦-1.json
API call for 𠢤 completed in 17.598721027374268 sec, 
 output written to deepseek/𠢤-1.json
API call for 勥 completed in 13.186040878295898 sec, 
 output written to deepseek/勥-1.json
API call for 𠢕 completed in 19.80334997177124 sec, 
 output written to deepseek/𠢕-1.json
API call for 𦌭 completed in 17.394242525100708 sec, 
 output written to deepseek/𦌭-1.json
API call for 𤃭 completed in 16.08709955215454 sec, 
 output written to deepseek/𤃭-1.json
API call for 𦊍 completed in 16.61062216758728 sec, 
 output written to deepseek/𦊍-1.json
API call for 𦌬 completed in 14.008094310760498 sec, 
 output written to deepseek/𦌬-1.json
API call for 𦊾 completed in 15.394119262695312 sec, 
 output written to deepseek/𦊾-1.json
API call for 䍙 completed in 11.869670629501343 sec, 
 output written to deepseek/䍙-1.json
API call for 叜 completed in 13.835052013397217 sec, 
 output written to deepseek/叜-1.json
API call for 

API call for 隺 completed in 14.470439195632935 sec, 
 output written to deepseek/隺-1.json
API call for 尞 completed in 14.624329328536987 sec, 
 output written to deepseek/尞-1.json
API call for 粦 completed in 15.197243690490723 sec, 
 output written to deepseek/粦-1.json
API call for 㣎 completed in 13.72849726676941 sec, 
 output written to deepseek/㣎-1.json
API call for 惢 completed in 17.012468814849854 sec, 
 output written to deepseek/惢-1.json
API call for 氾 completed in 13.499279022216797 sec, 
 output written to deepseek/氾-1.json
API call for 氶 completed in 14.214571714401245 sec, 
 output written to deepseek/氶-1.json
API call for 㕙 completed in 23.6403067111969 sec, 
 output written to deepseek/㕙-1.json
API call for 圼 completed in 13.461227416992188 sec, 
 output written to deepseek/圼-1.json
API call for 罙 completed in 17.084150552749634 sec, 
 output written to deepseek/罙-1.json
API call for 洰 completed in 12.680172443389893 sec, 
 output written to deepseek/洰-1.json
API call for 

In [37]:
!cd 

C:\Users\p2p2l\projects\wgong\zistory\zinets\dev\notebook
